In [ ]:
# Импортируем необходимые библиотеки

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

import xgboost as xgb
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
# Загрузка CSV-файла с компьютера

from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

import io
data = pd.read_csv(io.StringIO(uploaded['data_19-20.csv'].decode('utf-8')))

In [ ]:
# ИЛИ

In [ ]:
# Загрузка CSV-файла с гугл диска

from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data_19-20.csv', encoding = 'utf-8')

#data = pd.DataFrame(data)

In [ ]:
#data.head()
data

In [ ]:
# Проверка категориальности
print(data.dtypes)

# Убираем ненужные признаки
data = data.drop(['Код_ученика'], axis=1)
print(data.dtypes)

# Проверка пропущенных значений
print(data.isna().sum())

# Если вдруг есть пропущенные значения
# data['Age'].fillna(data['Age'].median(), inplace=True)
# data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

In [ ]:
data

In [ ]:
# Кодирование
encoder = LabelEncoder()

data['ЛогинОО'] = encoder.fit_transform(data['ЛогинОО'])

for col in ('Класс', 'Код_предмета', 'Балл', 'Отметка'):

  #data[col] = data[col].str.replace(' ', '')
  #data[col] = data[col].str.replace('\t', '')
  #data[col] = data[col].str.replace('\n', '')

  #data[col] = pd.to_numeric(data[col], errors='raise')

  #data[col] = data[col].str.replace(' ', '')
  #data[col] = data[col].astype(int)

print(data.dtypes)
data

In [ ]:
# Разделение параметров и меток
features = data.drop(['Балл', 'Отметка'], axis=1)
labels_1 = data['Балл']
labels_2 = data['Отметка']

# train-test (85% - обучение, 15% - валидация)
x_train_1, x_val_1, y_train_1, y_val_1 = train_test_split(features, labels_1, train_size=0.85)
x_train_2, x_val_2, y_train_2, y_val_2 = train_test_split(features, labels_2, train_size=0.85)

d_train_1 = xgb.DMatrix(x_train_1, y_train_1)
d_test_1 = xgb.DMatrix(x_val_1, y_val_1)

d_train_2 = xgb.DMatrix(x_train_2, y_train_2)
d_test_2 = xgb.DMatrix(x_val_2, y_val_2)

In [ ]:
# Настройка параметров

params_0 = {
    'tree_method': 'approx', # 'auto' не подходит, слижком большой датасет
    'booster': 'gbtree',
    'max_depth': 5, 
    'learning_rate': 0.1,
    'sample_type': 'uniform',
    'normalize_type': 'tree',
    'objective': 'reg:squarederror', # регрессия с квадратом потерь
    'rate_drop': 0.1,
    'n_estimators': 500
}

In [ ]:
xgb_clf_1 = xgb.train(params_0, d_train_1)
xgb_clf_2 = xgb.train(params_0, d_train_2)

In [ ]:
# Визуализация важности параматров

plt.figure(figsize=(16, 12))
xgb.plot_importance(xgb_clf_1)
plt.show()

plt.figure(figsize=(16, 12))
xgb.plot_importance(xgb_clf_2)
plt.show()

In [ ]:
# Прогноз
preds_1 = xgb_clf_1.predict(d_test_1)
preds_2 = xgb_clf_2.predict(d_test_2)
#print(preds)

# Оценка точности
print(np.round(accuracy_score(y_val_1, preds_1)*100, 2), '%')
print(np.round(accuracy_score(y_val_2, preds_2)*100, 2), '%')

In [ ]:
# Кросс-валидация k-fold

# DMatrix, состоящая из всех обучающих данных
dmatrix_data_1 = xgb.DMatrix(features, labels_1)
dmatrix_data_2 = xgb.DMatrix(features, labels_2)

# Параметры
params = {
    'objective': 'reg:squarederror', # регрессия с квадратом потерь
    'colsample_bytree': 0.3,
    'learning_rate': 0.1,
    'max_depth': 5,
}

cross_val_1 = xgb.cv(
    params=params,
    dtrain=dmatrix_data_1, 
    nfold=3,
    num_boost_round=50,
    early_stopping_rounds=10,
    metrics='error',
    as_pandas=True,
    seed=42)
cross_val_2 = xgb.cv(
    params=params,
    dtrain=dmatrix_data_2, 
    nfold=3,
    num_boost_round=50,
    early_stopping_rounds=10,
    metrics='error',
    as_pandas=True,
    seed=42)

print(cross_val_1.head())
print(cross_val_2.head())